FIAP

# TIAO - Machine Learning - Aula 03

## Comparação entre GridSearch e RandomSearch

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Carregar os dados
dados = pd.read_csv("https://tinyurl.com/42rvaw4p")
dados.sample(3)

In [ ]:
# Ajustar os dados
dados["smoker"] = dados["smoker"].apply(lambda x: 1 if x=="yes" else 0)
dados["sex"] = dados["sex"].apply(lambda x: 1 if x=="female" else 0)
del dados["region"]
dados.sample(10)

In [ ]:
# Divisão de TREINO e TESTE (holdout)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

outcome = dados["charges"] # Label
del dados["charges"]       # Retirei o label, então tudo o que sobra é feature

X_train, X_test, y_train, y_test = train_test_split(
    dados,
    outcome,
    test_size=0.3
)

In [ ]:
# Padronização dos dados
ss = StandardScaler()
ss.fit(X_train)
X_train = ss.transform(X_train)
X_test = ss.transform(X_test)

In [ ]:
# Imports para treinar o modelo de regressão
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor

In [ ]:
# Valores possíveis do hiperparâmetro "n_estimator" (tamanho da floresta)
n_estimators = [int(x) for x in np.linspace(100, 200, num=200)]
print(n_estimators)

In [ ]:
# Valores a serem investigados para outros hiperparâmetros
max_features = [None, "sqrt"]
max_depth = [int(x) for x in np.linspace(5, 30, num=15)]
min_samples_split = [2, 5, 10, 20]
min_samples_leaf = [1, 2, 4, 5, 10]

In [ ]:
# Criar o grid de parâmetros
params = {
    "n_estimators": n_estimators,
    "max_features": max_features,
    "max_depth": max_depth,
    "min_samples_split": min_samples_split,
    "min_samples_leaf": min_samples_leaf
}

In [ ]:
# Instância do estimador (modelo preditivo)
rf = RandomForestRegressor(random_state=42)

In [ ]:
# ESTRATÉGIA 1 - Grid Search
grid_rf = GridSearchCV(
    estimator=rf,
    param_grid=params,
    cv=3,
    verbose=2,
    n_jobs=-1  # Paraleliza para todos os cores do computador
)

In [ ]:
grid_rf.fit(X_train, y_train)

In [ ]:
# ESTRATÉGIA 2 - Random Search
random_rf = RandomizedSearchCV(   # mudei esta linha
    estimator=rf,
    param_distributions=params,   # mudei esta linha
    cv=3,
    verbose=2,
    n_jobs=-1,  # Paraleliza para todos os cores do computador
    random_state=42,              # mudei esta linha
    n_iter=20                     # mudei esta linha
)

In [ ]:
random_rf.fit(X_train, y_train)

In [ ]:
# Como pegar os melhores hiperparâmetros?
random_rf.best_params_

In [ ]:
# Como pegar o melhor modelo?
random_rf.best_estimator_

In [ ]:
# Como prever dados de teste com o melhor modelo encontrado?
# m = RandomForestRegressor(max_depth=17, max_features=None, min_samples_leaf=10,
#                      min_samples_split=10, n_estimators=167, random_state=42)
m = random_rf.best_estimator_
m.fit(X_train, y_train)
y_pred = m.predict(X_test)
y_pred

# Pipelines do Sklearn

In [ ]:
data = pd.read_csv("https://raw.githubusercontent.com/MicrosoftDocs/ml-basics/master/data/daily-bike-share.csv")
data.sample(5)

In [ ]:
# Remover a coluna "dteday" para evitar erro de processamento
del data["dteday"]

In [ ]:
# Verificar se há dados outliers
from sklearn.neighbors import LocalOutlierFactor

lof = LocalOutlierFactor()
outliers = lof.fit_predict(data)
pd.DataFrame(outliers).value_counts()

In [ ]:
# Remover os outliers
data["outliers"] = outliers
data = data[ data["outliers"] > 0 ]

In [ ]:
data.shape

In [ ]:
# Filtrar colunas de interesse
data = data[["season", "mnth", "holiday", "weekday", "workingday",
            "weathersit", "temp", "atemp", "hum", "windspeed", "rentals"]]

In [ ]:
# Separar dados e labels, e depois em TREINO e TESTE
from sklearn.model_selection import train_test_split

X = data.drop(columns=["rentals"])
y = data["rentals"]

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=123
)

## Transformação de dados

In [ ]:
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [ ]:
# Processo dados numéricos
numeric_transform = Pipeline(steps=[
    ("imputar", SimpleImputer(strategy="mean")),
    ("padronizar", StandardScaler())
])

# Processo dados categoricos ordinais
categorical_transform = Pipeline(steps=[
    ("imputar", SimpleImputer(strategy="most_frequent")),
    ("encoder", OrdinalEncoder())
])

In [ ]:
# Crias as listas de features
numeric_features = ["temp", "atemp", "hum", "windspeed"]
categorical_features = ["season", "mnth", "holiday", "weekday", "workingday", "weathersit"]

In [ ]:
# Aplicar o preprocessamento dos dados
preprocessor = ColumnTransformer(
  transformers=[
      ("numerica", numeric_transform, numeric_features), # (nome, transformação, lista de feats)
      ("categorica", categorical_transform, categorical_features)
  ]
)

## Cria o pipeline e executa-o, avaliando o modelo

In [ ]:
# Definir o pipeline
pipeline = Pipeline(steps=[
    ("preprocessamento", preprocessor),
    ("regressor", RandomForestRegressor(random_state=42))
])

In [ ]:
# Faz o treinamento
rf_model = pipeline.fit(X_train, y_train)
print(rf_model)

In [ ]:
# Avaliar o pipeline
from sklearn.metrics import r2_score

preds = rf_model.predict(X_test)
print( r2_score(y_test, preds) )

## Usando Pipelines em Cross Validation

Este passo serve para qualquer preditor.

In [ ]:
from sklearn.model_selection import cross_validate

scores = cross_validate(pipeline, # isso pode ser substituído por qualquer modelo preditivo
                        X_train,
                        y_train,
                        cv=5)

In [ ]:
scores

In [ ]:
# Reportar o resultado final
media = scores["test_score"].mean()
std = scores["test_score"].std()

print(f"Score: {media:.2f} ± {std:.2f}")

# Exportação / Importação de modelos

In [ ]:
# Exportar o modelo RF - No notebook de criação do modelo
import joblib

joblib.dump(rf_model, "./modelo_RF.pkl")

['./modelo_RF.pkl']

In [ ]:
# Importar o modelo - No ambiente de utilização do modelo (ex: servidor)
m_final = joblib.load("modelo_RF.pkl")
m_final

Pipeline(steps=[('preprocessamento',
                 ColumnTransformer(transformers=[('numerica',
                                                  Pipeline(steps=[('imputar',
                                                                   SimpleImputer()),
                                                                  ('padronizar',
                                                                   StandardScaler())]),
                                                  ['temp', 'atemp', 'hum',
                                                   'windspeed']),
                                                 ('categorica',
                                                  Pipeline(steps=[('imputar',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OrdinalEncoder())]),
                                                  ['season', 'mnth', 'holiday',
                                                   'weekday', 'workingday',
                                                   'weathersit'])])),
                ('regressor', RandomForestRegressor(random_state=42))])

In [ ]:
# Fazer novas predições - ATENÇÃO, ISSO NÃO RODA!
novos_dados = pd.read_csv("... novos dados...")
novas_predicoes = m_final.predict( novos_dados )